Data Science Capstone Project Week 3 

Download and import necessaary libraries

In [123]:
# import needed libraries
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import csv
#!conda install -c conda-forge folium=0.5.0 --yes 
from pandas.io.json import json_normalize
import folium
import numpy as np
from sklearn.cluster import KMeans
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
import functools


Foursquare Credentials

In [124]:
CLIENT_ID = 'OEKMEBXITJT02VBZB3LP4ATYZ35OI3PLISMSZUTSJSFMBZQI' # your Foursquare ID
CLIENT_SECRET = 'POCWPILITAHTRCVJB1EKV1Y1OM30PZXOGVYAPZ4CCPZKHMUQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: OEKMEBXITJT02VBZB3LP4ATYZ35OI3PLISMSZUTSJSFMBZQI
CLIENT_SECRET:POCWPILITAHTRCVJB1EKV1Y1OM30PZXOGVYAPZ4CCPZKHMUQ


In [125]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1008658788"
#url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_content = requests.get(url).text
soup = bs(html_content, "lxml")


In [126]:
table = soup.find("table", attrs={"class": "wikitable"})
#print(soup)
# identify columns
th = table.find_all("th")
headers = []
for x in th:
    headers.append(x.text.strip())

# identify rows  
tr = table.find_all("tr")
rows = []

# split elements into lists and remove empty elements
for x in tr[1:]:
    rows.append(x.text.split("\n"))
for x in rows:
    for y in x:
        x.remove('')

In [127]:
df = pd.DataFrame(rows, columns = headers) 
df = df[df.Borough != 'Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [128]:
df.shape

(103, 3)

In [129]:
#Download and read the csv file containing coordinates 
url = "https://cocl.us/Geospatial_data"
coords = pd.read_csv(url)

In [130]:
#Merge coordinates with boroughs dataframe
df_coords= df.merge(coords, on="Postal Code")
df_coords.head(20)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [131]:
neighborhood_latitude=df_coords.loc[0, 'Latitude'] 
neighborhood_longitude=df_coords.loc[0, 'Longitude']

In [132]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL



url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=OEKMEBXITJT02VBZB3LP4ATYZ35OI3PLISMSZUTSJSFMBZQI&client_secret=POCWPILITAHTRCVJB1EKV1Y1OM30PZXOGVYAPZ4CCPZKHMUQ&v=20180605&ll=43.7532586,-79.3296565&radius=500&limit=100'

In [133]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '607fe3f133221c3b49b6a155'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [134]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114


In [136]:
if "cached_requests_get" not in dir():
    @functools.lru_cache(maxsize=10000)
    def cached_requests_get(url):
        for _ in range(5):
            response = requests.get(url)
            json = response.json()
            if json["meta"]["code"] == 500:
                time.sleep(5)
            else:
                return response


In [137]:
venues_list=[]
LIMIT = 100
radius = 1000 


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    global venues_list
    venues_list=[]
    
   
    for name, lat, lng in zip(names, latitudes, longitudes):

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        request_json = cached_requests_get(url).json()
        results = request_json["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [138]:
Neighborhood_venues = getNearbyVenues(names=df_coords['Neighbourhood'],
                                   latitudes=df_coords['Latitude'],
                                   longitudes=df_coords['Longitude']
                                    )

In [139]:
#display(venues_list)
#Neighborhood_venues.loc[Neighborhood_venues['Neighborhood'] == 'Islington Avenue, Humber Valley Village']

In [140]:
# getting dummies

Toronto_dummies = pd.get_dummies(Neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")
Toronto_dummies = Toronto_dummies.drop('Neighborhood', axis=1)
Toronto_dummies.insert(1,'Neighborhood',Neighborhood_venues['Neighborhood'] )

# grouping dataframe
Toronto_grouped = Toronto_dummies.groupby('Neighborhood').mean().reset_index()
#Toronto_grouped.head()
#Toronto_grouped.get('Neighborhood')

#work-around to avoid the key error NO NEED ANYMORE
#first_column = Toronto_grouped.iloc[:, 0]
#Toronto_grouped['Neighborhood'].head()
#first_column.head()

In [141]:
# sort in descending order function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [142]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

#workaround to avoid key error NO NEED ANYMORE
#neighborhoods_venues_sorted['Neighborhood'] = first_column

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Escape Room,Dessert Shop
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Dance Studio,Playground,Pub,Coffee Shop,Airport Terminal,College Rec Center,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Sandwich Place,Middle Eastern Restaurant,Ice Cream Shop,Supermarket,Restaurant,Deli / Bodega,Diner
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Yoga Studio,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant
4,"Bedford Park, Lawrence Manor East",Sushi Restaurant,Coffee Shop,Italian Restaurant,Sandwich Place,Liquor Store,Cupcake Shop,Indian Restaurant,Butcher,Café,Pub


In [143]:
# set number of clusters
kclusters = 10

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([7, 5, 7, 7, 7, 7, 7, 7, 7, 7])

In [144]:
# add clustering labels, clean up dataframe
neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)
Toronto_merged = df_coords.copy()
Toronto_merged.rename(columns={'Neighbourhood': 'Neighborhood'},inplace=True)
#Toronto_merged.drop('Postal Code',axis=1,inplace=True)


# merge clusters and venues
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged['Cluster Labels'].replace(float("NaN"),kclusters,inplace = True)
Toronto_merged.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,5.0,Portuguese Restaurant,Pizza Place,Coffee Shop,Hockey Arena,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,7.0,Coffee Shop,Park,Café,Bakery,Breakfast Spot,Restaurant,Pub,Theater,French Restaurant,Hotel
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,7.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Boutique,Vietnamese Restaurant,Miscellaneous Shop,Dog Run,Diner,Discount Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,7.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Discount Store,Sandwich Place,Park,Mexican Restaurant,Japanese Restaurant,Italian Restaurant,Fried Chicken Joint
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,3.0,Fast Food Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
7,M3B,North York,Don Mills,43.745906,-79.352188,7.0,Gym,Coffee Shop,Restaurant,Café,Dim Sum Restaurant,Chinese Restaurant,Sandwich Place,Bike Shop,Beer Store,Sporting Goods Shop
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,7.0,Pizza Place,Pharmacy,Gastropub,Breakfast Spot,Flea Market,Intersection,Bank,Athletics & Sports,Pet Store,Gym / Fitness Center
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,7.0,Clothing Store,Coffee Shop,Café,Bubble Tea Shop,Cosmetics Shop,Japanese Restaurant,Middle Eastern Restaurant,Theater,Lingerie Store,Electronics Store


In [145]:
# Find Toronto

address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i + x + (i*x)**2 for i in range(kclusters+1)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    if cluster.is_integer():
        cluster = int(cluster)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [149]:
#print(Toronto_merged['Cluster Labels'] == "1")
#Toronto_merged.loc['Cluster Labels'] == "1"]

0      False
1      False
2      False
3      False
4      False
       ...  
98     False
99     False
100    False
101    False
102    False
Name: Cluster Labels, Length: 103, dtype: bool


C:\Users\alexg\Anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [162]:
Cluster_1=Toronto_merged[Toronto_merged['Cluster Labels']==1]
Cluster_1.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
21,M6E,York,Caledonia-Fairbanks,43.689026,-79.453512,1.0,Park,Women's Store,Pool,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.685347,-79.338106,1.0,Convenience Store,Park,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
61,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,1.0,Park,Bus Line,Swim School,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Yoga Studio
64,M9N,York,Weston,43.706876,-79.518188,1.0,Jewelry Store,Park,Convenience Store,Yoga Studio,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
